In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt-get update
!apt-get install -y unrar

In [ ]:
!unrar x "/content/drive/MyDrive/teknofest_2/dataset.rar"


In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import yaml
import cv2
import matplotlib.pyplot as plt

In [ ]:
!yolo train model=yolo12l.pt data=/content/drive/MyDrive/teknofest_2/data.yaml epochs=150 imgsz=640

In [ ]:
!yolo detect predict model=/content/drive/MyDrive/teknofest_2/best12v.pt source=/content/dataset/val/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

In [ ]:
# Örnek video yolu (kendi yoluna göre düzenle)
video_path = "/content/drive/MyDrive/teknofest_2/video2.MP4"

# Eğitimli YOLOv8 modelin yolu
model_path = "/content/drive/MyDrive/teknofest_2/best12v.pt"

In [ ]:
import cv2
import time
import random
import os
from ultralytics import YOLO
from IPython.display import Video, display

In [ ]:
def get_name(file_path):
    return os.path.basename(file_path)

def get_save_path(file_name, folder_name="results"):
    save_dir = os.path.join("/content", folder_name)
    os.makedirs(save_dir, exist_ok=True)
    return os.path.join(save_dir, file_name)

def draw_box(img, result, class_list, colors, label_size=1.0):
    xyxy = result.boxes.xyxy.cpu().numpy()
    conf = result.boxes.conf.cpu().numpy()
    cls_ids = result.boxes.cls.cpu().numpy().astype(int)
    for cls_id, c, box in zip(cls_ids, conf, xyxy):
        pt1, pt2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
        color = colors[cls_id]
        label = f"{class_list[cls_id]} {c:.2f}"
        cv2.rectangle(img, pt1, pt2, color, 2)
        cv2.putText(img, label, (pt1[0], pt1[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, label_size, (255,255,255), 2)
    return img

def draw_fps(image, fps):
    cv2.putText(image, f"FPS: {fps:.2f}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    return image

In [ ]:
def detect_and_save_video(video_path, model_path, output_folder="results", img_size=640, label_size=1.0):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(video_path)

    class_list = model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in class_list]

    width = int(cap.get(3))
    height = int(cap.get(4))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps_out = cap.get(cv2.CAP_PROP_FPS)

    output_file = get_save_path(get_name(video_path), output_folder)
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), fps_out, (width, height))

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        start = time.time()
        results = model.predict(frame, imgsz=img_size, device=0, verbose=False)
        result = results[0]
        frame = draw_box(frame, result, class_list, colors, label_size)
        end = time.time()
        fps = 1 / (end - start)

        frame = draw_fps(frame, fps)
        out.write(frame)
        frame_idx += 1
        print(f"({frame_idx}/{total_frames}) Frame işlendi.")

    cap.release()
    out.release()
    print("Video kaydedildi:", output_file)
    return output_file

In [ ]:
output_path = detect_and_save_video(video_path, model_path, output_folder="yolo_results", img_size=640)
display(Video(output_path, embed=True))